In [1]:
## Julia Cope
## read in csv from 01 file that contains all the articles and text
## use word embeddings from word2vec and doc2vec to make the document embeddings
## cluster the documents to see if there is something interesting...


In [2]:
#!pip install xgboost

In [3]:
import pandas as pd
import gensim
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
%matplotlib inline 
from sklearn.manifold import TSNE

C:\Users\julia\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\julia\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\julia\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe

In [4]:
## https://github.com/RaRe-Technologies/gensim-data
## https://towardsdatascience.com/using-word2vec-to-analyze-news-headlines-and-predict-article-success-cdeda5f14751 


import gensim.downloader as api

info = api.info()  # show info about available models/datasets
model = api.load("word2vec-google-news-300")  # download the model and return as object ready for use
model.most_similar("cat")

[('cats', 0.8099379539489746),
 ('dog', 0.760945737361908),
 ('kitten', 0.7464985251426697),
 ('feline', 0.7326234579086304),
 ('beagle', 0.7150582671165466),
 ('puppy', 0.7075453400611877),
 ('pup', 0.6934291124343872),
 ('pet', 0.6891531348228455),
 ('felines', 0.6755931973457336),
 ('chihuahua', 0.6709762215614319)]

In [5]:
## make document embeddings
## cluster the document embeddings



In [6]:
PressR_df = pd.read_csv("01_pressreleases.csv", encoding='latin-1')
PressR_df = PressR_df.drop(columns=['Unnamed: 0', 'Length', "Section", "Author"])
PressR_df.head()
PressR_df_input = PressR_df

In [31]:
PressR_df = PressR_df_input

In [7]:
# Grab all the titles 
article_titles = PressR_df['Article']# Create a list of strings, one for each title
titles_list = [title for title in article_titles]

# Collapse the list of strings into a single long string for processing
big_title_string = ' '.join(titles_list)

from nltk.tokenize import word_tokenize

# Tokenize the string into words
tokens = word_tokenize(big_title_string)

# Remove non-alphabetic tokens, such as punctuation
words = [word.lower() for word in tokens if word.isalpha()]

# Filter out stopwords
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

words = [word for word in words if not word in stop_words]

# Print first 10 words
words[:10]

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


['cairn',
 'pleased',
 'announce',
 'appointment',
 'catherine',
 'krajicek',
 'independent',
 'director',
 'effect',
 'july']

In [8]:
model.most_similar("Chevron")
#model.vector_size

[('ChevronTexaco', 0.8098915219306946),
 ('ExxonMobil', 0.7666822671890259),
 ('Exxon', 0.7464423179626465),
 ('Exxon_Mobil', 0.737562894821167),
 ('Chevron_Corp.', 0.712766170501709),
 ('Shell', 0.68023282289505),
 ('ConocoPhillips', 0.6716794371604919),
 ('Chevron_CVX', 0.66364586353302),
 ('Chevron_Texaco', 0.6607154011726379),
 ('Conoco', 0.6589248180389404)]

In [9]:
# Filter the list of vectors to include only those that Word2Vec has a vector for
# words has my data in it...
vector_list = [model[word] for word in words if word in model.key_to_index]

# Create a list of the words corresponding to these vectors
words_filtered = [word for word in words if word in model.key_to_index]

# Zip the words together with their vector representations
word_vec_zip = zip(words_filtered, vector_list)

# Cast to a dict so we can turn it into a DataFrame
word_vec_dict = dict(word_vec_zip)
df = pd.DataFrame.from_dict(word_vec_dict, orient='index')
df.head(7)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
cairn,0.087891,0.010071,-0.033691,0.328125,0.312500,-0.211914,-0.046631,0.052734,0.191406,-0.083984,...,0.234375,-0.220703,-0.441406,0.292969,0.031738,-0.211914,-0.070801,0.123535,0.219727,0.117188
pleased,-0.207031,0.251953,-0.014832,-0.129883,0.224609,0.146484,0.205078,-0.051758,0.193359,0.156250,...,-0.088867,-0.014282,0.044678,-0.010620,0.096680,0.072754,0.037598,0.116699,0.033447,-0.373047
announce,-0.271484,-0.052734,-0.010925,-0.127930,-0.022583,-0.152344,0.107422,-0.062500,0.416016,0.086426,...,0.250000,-0.007355,0.038574,-0.102539,0.013428,-0.345703,-0.002930,0.226562,0.020386,-0.223633
appointment,-0.197266,-0.227539,-0.084961,-0.247070,-0.056152,-0.056152,0.065430,-0.523438,0.117188,0.182617,...,-0.143555,0.157227,-0.275391,-0.036377,0.075684,0.217773,0.024902,0.097168,-0.218750,-0.020020
catherine,-0.188477,-0.086914,-0.038086,0.156250,0.005585,0.018921,0.086426,-0.078125,-0.152344,0.033203,...,0.214844,0.020264,0.018799,-0.009155,-0.061279,-0.142578,0.008179,-0.061279,0.121582,0.306641
independent,-0.283203,-0.251953,-0.116699,0.043457,-0.092773,0.163086,0.052979,-0.123047,0.029785,0.133789,...,-0.063965,0.080566,-0.210938,0.185547,-0.128906,-0.089844,0.037109,0.287109,-0.210938,-0.059814
director,-0.040283,-0.322266,0.040527,-0.129883,-0.067383,0.036377,0.246094,-0.065918,-0.070312,-0.205078,...,-0.148438,0.111816,0.036865,0.204102,0.051270,-0.229492,0.285156,0.181641,0.300781,0.205078


In [10]:
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in model.key_to_index]
    return np.mean(model[doc], axis=0)

# Our earlier preprocessing was done when we were dealing only with word vectors
# Here, we need each document to remain a document 
def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] 
    return doc

# Function that will help us drop documents that have no word vectors in word2vec
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.key_to_index for word in doc)

# Filter out documents
def filter_docs(corpus, texts, condition_on_doc):
    """
    Filter corpus and texts given the function condition_on_doc which takes a doc. The document doc is kept if condition_on_doc(doc) is true.
    """
    number_of_docs = len(corpus)

    if texts is not None:
        texts = [text for (text, doc) in zip(texts, corpus)
                 if condition_on_doc(doc)]

    corpus = [doc for doc in corpus if condition_on_doc(doc)]

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts)

In [11]:
# Preprocess the corpus
corpus = [preprocess(title) for title in titles_list]

# Remove docs that don't include any words in W2V's vocab
corpus, titles_list = filter_docs(corpus, titles_list, lambda doc: has_vector_representation(model, doc))

# Filter out any empty docs
corpus, titles_list = filter_docs(corpus, titles_list, lambda doc: (len(doc) != 0))
x = []
for doc in corpus: # append the vector for each document
    x.append(document_vector(model, doc))
    
X = np.array(x) # list to array

0 docs removed
0 docs removed


In [12]:
PressR_df.head(10)

,Newspaper,Date,Headline,Article
0,ENP Newswire,2019-04-30,-Cairn Energy PLC - Appointment of Non-Executi...,Cairn is pleased to announce the appointment...
1,ENP Newswire,2018-05-07,-Calima Energy - Calima launches friendly take...,In what looks to be an astute corporate play...
2,ENP Newswire,2018-07-09,-Calima Energy Limited - Company Presentation,Calima Energy Limited (ASX:CE1) ('Calima' or...
3,ENP Newswire,2021-07-22,-Canadian Solar Inc. - Recurrent Energy Comple...,GUELPH - Canadian Solar Inc. ('Canadian Sola...
4,ENP Newswire,2019-11-26,-Cenovus Energy Selects IBM to Boost Digital T...,Implementation of SAP S/4HANA paves the way ...
5,M2 PressWIRE,2022-10-07,-Chevron - calbio expand partnership on dairy ...,"October 7, 2022 Release date- 06102022 - SA..."
6,ENP Newswire,2021-09-08,"-Chevron, Delta Air Lines, and Google Announce...","SAN RAMON, MOUNTAIN VIEW, Calif./ATLANTA, Ga..."
7,M2 PressWIRE,2022-11-08,-Chevron and JERA advance lower carbon solutio...,"November 8, 2022 Release date- 07112022 - H..."
8,ENP Newswire,2022-03-02,-Chevron Executing Plans to Deliver Higher Ret...,Chevron Executing Plans to Deliver Higher Re...
9,ENP Newswire,2022-12-16,-Chevron Invests in Carbon Capture and Removal...,"HOUSTON VANCOUVER, British Columbia - Chevro..."


In [79]:
#corpus


In [14]:
type(X)


numpy.ndarray

In [15]:
from sklearn.datasets import load_iris

iris = load_iris()
xyz = iris.data

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\datasets\base.py:241: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  target = np.empty((n_samples,), dtype=np.int)
C:\Users\julia\Anaconda3\lib\site-packages\sklearn\datasets\base.py:245: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, 

In [61]:
from sklearn.cluster import AgglomerativeClustering
#cluster_model = AgglomerativeClustering(distance_threshold=0.99, n_clusters=None)
cluster_model = AgglomerativeClustering( n_clusters=20)

In [62]:
cluster_model = cluster_model.fit(X)

In [63]:
# Number of clusters
cluster_model.n_clusters_

20

In [64]:
# clusters
#cluster_model.labels_

In [65]:
PressR_df = PressR_df_input

In [66]:
clusters = pd.DataFrame(cluster_model.labels_)
PressR_df = pd.concat([PressR_df, clusters], axis=1)
PressR_df = PressR_df.rename(columns={0: "cluster"})
doc_embeddings = pd.DataFrame(X)
PressR_df = pd.concat([PressR_df, doc_embeddings], axis=1)

In [67]:

PressR_df.groupby(['cluster'])['cluster'].count()

cluster
0      93
1     118
2      36
3      75
4      42
5      62
6      87
7      33
8      44
9      23
10     34
11     61
12     27
13     17
14      4
15     32
16     26
17     41
18     25
19     33
Name: cluster, dtype: int64

In [84]:
#http://songhuiming.github.io/pages/2017/04/02/jupyter-and-pandas-display/
    
PressR_df[PressR_df["cluster"] == 0][{"Headline","Article"}]

Article  \
3      GUELPH - Canadian Solar Inc. ('Canadian Solar' or 'the Company') (NASDAQ: CSIQ) today announced its wholly-owned subsidiary, Recurrent Energy, LLC ('Recurrent Energy'), completed the sale of the Maplewood 1 and Maplewood 2 solar projects to a leading U.S. annuity and life insurance company.  Both projects are located in Pecos County, Texas, near the towns of Fort Stockton and McCamey, and have a combined capacity of 328 MWp or 250 MWac. Maplewood 1 and Maplewood 2 are both currently under construction and are expected to reach commercial operation before the end of Q3 2021.  Construction of the Maplewood 1 and Maplewood 2 solar projects commenced in the summer of 2020 after Recurrent Energy closed $ 282 million of debt financing provided by a bank club led by Norddeutsche Landesbank and included Export Development Canada, Rabobank, National Bank of Canada and Bayerische Landesbank. The Maplewood projects' offtakers are Anheuser-Busch, the world's largest brewer, and Energy Transfer Partners, the Dallas-based Fortune 100 oil and gas pipeline company. Anheuser-Busch will purchase the 222 MWac from the Maplewood 1 project under a 15-year power purchase agreement ('PPA'), helping the brewer achieve its 100% renewable electricity four years in advance of its 2025 target. Maplewood 2 will deliver 28 MWac of clean power to Energy Transfer Partners under a 15-year PPA.  Both projects utilize Canadian Solar's high-efficiency bifacial modules. The electricity generated by the Maplewood projects is expected to displace approximately 518,000 metric tons of carbon dioxide emissions annually, equivalent to taking 112,655 cars off the road.  'This transaction is Recurrent's first major project sale with commercial and industrial (CI) offtakers,' said Shawn Qu, Chairman and CEO of Canadian Solar. 'We already see that an increasing number of corporations and public sector organizations across the globe are directly contracting with renewable energy projects to serve their load. In particular, corporates are finding that solar energy can not only reduce their electricity costs but can also help them meaningfully reduce the carbon footprint of their operations. This is a particularly exciting milestone given the ongoing challenges posed by the global pandemic and is a testament that solar energy has truly entered into the mainstream.'  ABOUT CANADIAN SOLAR INC.  Canadian Solar was founded in 2001 in Canada and is one of the world's largest solar technology and renewable energy companies. It is a leading manufacturer of solar photovoltaic modules, provider of solar energy and battery storage solutions, and developer of utility-scale solar power and battery storage projects with a geographically diversified pipeline in various stages of development. Over the past 20 years, Canadian Solar has successfully delivered over 55 GW of premium-quality, solar photovoltaic modules to customers across the world. Likewise, since entering the solar project development business in 2010, Canadian Solar has developed, built and connected over 5.7 GWp in over 20 countries across the world. Currently, the Company has around 500 MWp of projects in operation, nearly 6 GWp of projects under construction or in backlog (late-stage), and an additional 15 GWp of projects in pipeline (mid- to early- stage). Additionally, Canadian Solar has 1.2 GWh of battery storage projects under construction, and nearly 17 GWh of battery storage projects in backlog or pipeline. Canadian Solar is one of the most bankable companies in the solar and renewable energy industry, having been publicly listed on the NASDAQ since 2006.  ABOUT RECURRENT ENERGY (CANADIAN SOLAR SUBSIDIARY)  Recurrent Energy is a leading utility-scale solar and storage project developer, delivering competitive, clean electricity to large energy buyers. Based in the U.S., Recurrent Energy is a wholly owned subsidiary of Canadian Solar Inc. and functions as Canadian Solar's U.S. project development arm

In [83]:
pd.set_option('display.max_colwidth', -1)
PressR_df[PressR_df["cluster"] == 1][{"Headline","Article"}]

C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


Article  \
10     Chevron Phillips Chemical Company LLC and QatarEnergy announced today that they are proceeding with the construction of an $ 8.5 billion integrated polymers facility in Orange, Texas, expected to create more than 500 full-time jobs and approximately 4,500 construction jobs and generate an estimated $ 50 billion for the community in residual economic impacts.  The companies have made a positive final investment decision on the project and created a joint venture company, Golden Triangle Polymers Company LLC, named for the Golden Triangle region of Texas that includes the city of Orange. Chevron Phillips Chemical owns a 51% equity share in the joint venture and QatarEnergy owns 49%.  'Chevron Phillips Chemical and QatarEnergy have collaborated for over 20 years on the assets we operate together in Qatar. We have a great relationship and a proven track record of operating these facilities safely and reliably,' said Chevron Phillips Chemical President and CEO Bruce Chinn. 'Our products help make life better for billions of people every day, and they are part of a lower carbon future. This facility will help meet the growing demand for our products and improve the quality of life for the world's growing global population.'  His Excellency Mr. Saad Sherida Al-Kaabi, the Minister of State for Energy Affairs, the President and CEO of QatarEnergy said: 'We are excited to announce taking the FID on our largest petrochemical investment ever, highlighting QatarEnergy's integrated position as a major player in the LNG and international exploration sectors, as well as being a global petrochemicals producer. This important project will complement QatarEnergy's growing portfolio, both internationally as well as in the United States, and will help meet growing global demand for polymers. It builds on our long-term and successful partnership with Chevron Phillips Chemical, and we look forward to further collaborations in the future.'  Once operational, the plant will produce Marlex polyethylene. Polyethylene is used in the production of durable goods like pipe for natural gas and water delivery and recreational products such as kayaks and coolers. It is also used in essential packaging applications to protect and preserve food, helping prevent it from going to landfills, and keep medical supplies sterile.  The plant, expected to begin operations in 2026, will include a 2,080 KTA ethane cracker and two 1,000 KTA high-density polyethylene units. The project is targeting to have approximately 25% lower greenhouse gas emissions than similar facilities in the United States and Europe, supporting the company's efforts to help enable a lower carbon future. Chevron Phillips Chemical will manage engineering, procurement and construction for the project and operate the facility after start-up.  Construction of the Golden Triangle Polymers plant will begin immediately near Chevron Phillips Chemical's existing facility in Orange, located 113 miles east of Houston. Chevron Phillips Chemical and its predecessors have had a presence in the Orange community since 1955.  The engineering, procurement and construction of the polyethylene units will be executed through ZDJV, a joint venture between Zachry Industrial Inc. (Zachry Group) and DL USA, Inc. The furnace portion of the ethane cracker engineering and procurement will be executed by T.EN Stone Weber Process Technology, Inc. while PCL Industrial Construction Co. will provide construction services. Engineering, procurement and construction for the additional portions of the ethane cracker will be executed by JKJV, a joint venture between JGC America, Inc. and Kiewit Energy Group, Inc., and BMZ Third Coast Partners, a joint venture between Burns McDonnell Engineering Company, Inc. and Zachry Industrial, Inc., (Zachry Group) will execute the utilities and infrastructure scope of work. The main automation contractor for the project is Emerson Process Management, LLLP, and W.T. Byler Co., Inc. 

In [105]:
#pd.reset_option("display")
pd.set_option('display.max_colwidth', None)

PressR_df[PressR_df["cluster"] == 16][{"Headline"}]

,Headline
34,-G2 Energy Provides Message from the President and announces a Private Placement
42,-Marathon Petroleum Corp. Announces Agreement for $ 21 Billion Sale of Speedway
63,-Marathon Petroleum Provides Update On Strategic Review To Enhance Shareholder Value
75,-Petrolia Energy Completes Acquisition of Bow Energy
81,-Quintana Energy Services Announces the Addition of Bobby S. Shackouls to Its Board of Directors
120,-Fitch: Strong Sponsors Benefit MLPs in Challenging Environment
212,CHESAPEAKE ENERGY CORPORATION APPOINTS JOSH VIETS EXECUTIVE VICE PRESIDENT AND CHIEF OPERATING OFFICER
312,"CVR Energy Reports Evaluation of Strategic Alternatives, Sale of Cushing Terminal"
345,Energy Transfer Equity Announces Quarterly Cash Distribution; Maintaining Distribution at 3Q 2015 level in light of current market environment; Extends management fee to ETP through 2016; Earnings to be released February 24
346,"Energy Transfer Equity to Combine with Williams; Anticipated Commercial Synergies Exceed $2 Billion of Incremental EBITDA by 2020; Up to $400 Million of Additional Cost Savings Expected from the Implementation of ETE's Shared Service Model; Williams' Stockholders Can Elect to Receive Shares Issued by New ETE C-corp and/or Cash, Subject to Proration If Either is Oversubscribed; Transaction is Immediately Accretive to Cash Flow and Distributions for Both ETE and WMB; Williams Partners L.P. (WPZ) to Retain Its Name and Remain Headquartered in Tulsa"
